# Object Storage에서 Parquet 파일 읽기 예제


## 1) 필요한 라이브러리 설치


In [ ]:
!pip install boto3 pyarrow pandas

## 2) AWS S3 클라이언트 생성

- `ACCESS_KEY`, `SECRET_KEY` 자리에 실제 환경 변수 값을 사용하세요.

In [ ]:
import boto3

ACCESS_KEY = "{AWS_ACCESS_KEY_ID}"
SECRET_KEY = "{AWS_SECRET_ACCESS_KEY}"

s3 = boto3.client(
    service_name="s3",
    region_name="kr-central-2",
    endpoint_url="https://objectstorage.kr-central-2.kakaocloud.com",
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)

## 3) Parquet 파일 키 목록 가져오기


In [ ]:
import io
import random
import pyarrow.parquet as pq
import pandas as pd

bucket_name = "data-catalog-bucket"
prefix = "data-catalog-dir/user_behavior_prediction_batch/"

resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
keys = [obj["Key"] for obj in resp.get("Contents", []) if obj["Key"].endswith(".parquet")]

if not keys:
    raise ValueError(f"No parquet files found under prefix {prefix!r}")

## 4) Parquet 파일 중 하나를 랜덤으로 선택


In [ ]:
random_key = random.choice(keys)

## 5) 선택한 Parquet 파일 다운로드 및 DataFrame 변환

1. S3에서 객체를 다운로드하여 메모리 버퍼(`BytesIO`)에 로드
2. PyArrow로 Parquet 테이블로 읽기
3. Pandas DataFrame으로 변환 후 상위 10개 행 출력

In [ ]:
obj = s3.get_object(Bucket=bucket_name, Key=random_key)
buf = io.BytesIO(obj["Body"].read())
table = pq.read_table(buf)
df = table.to_pandas()

df.head(10)